In [1]:
# imports
import os, sys
import io
import time
import numpy as np
import xarray as xr
import dask
import dask.array as da
import rasterio
import arcpy

# import tools
#sys.path.append('../../../shared') temp for demo
sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
import arc, tools, satfetcher

# import gdvspectra module
#sys.path.append('../../../modules') temp for demo
sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
import cog

# globals 
AWS_KEY = ''
AWS_SECRET = ''
STAC_ENDPOINT = 'https://explorer.sandbox.dea.ga.gov.au/stac/search'
RESULT_LIMIT = 250

In [2]:
# get minimum bounding geom from input 
bbox = [119.32132692558059, -22.774853394924776, 119.34873641649256, -22.745010351081362]

# get collections based on platform 
collections = ['s2a_ard_granule', 's2b_ard_granule']

# prepare start and end date times
in_from_date = '2020-01-01'
in_to_date = '2020-12-31'

in_slc_off = False

RESULT_LIMIT = 250

# fetch stac data 
feats = cog.fetch_stac_data(stac_endpoint=STAC_ENDPOINT, 
                            collections=collections, 
                            start_dt=in_from_date, 
                            end_dt=in_to_date, 
                            bbox=bbox,
                            slc_off=in_slc_off,
                            limit=RESULT_LIMIT)

Beginning STAC search for items. This can take awhile.
Searching collection: s2a_ard_granule
Searching collection: s2b_ard_granule
Sorting result by time (old to new).
Found a total of 148 scenes.


In [3]:
in_bands = 'Blue;Green;Red;NIR1;SWIR2;SWIR3;OA_Mask'
in_platform = 'Sentinel'

# prepare band (i.e. stac assets) names
assets = arc.prepare_band_names(in_bands=in_bands, 
                                in_platform=in_platform)

In [4]:
in_epsg = 3577
in_res = 10
in_snap = True

# convert raw stac into dict with coord reproject, etc.
arcpy.SetProgressorLabel('Converting STAC data into useable format...')
meta, asset_table = cog.prepare_data(feats, 
                                     assets=assets,
                                     bounds_latlon=bbox, 
                                     bounds=None, 
                                     epsg=in_epsg, 
                                     resolution=in_res, 
                                     snap_bounds=in_snap,
                                     force_dea_http=True)

Converting raw STAC data into numpy format.
Converted raw STAC data successfully.


In [5]:
in_resampling = 'Nearest'
in_fill_value = '-999'
in_chunk_size = -1
in_dtype = 'int16'
in_rescale = True

# prepare resample and fill value types
resampling = in_resampling.lower()
fill_value = arc.prepare_fill_value_type(in_fill_value)

# convert assets to dask array
darray = cog.convert_to_dask(meta=meta, 
                             asset_table=asset_table, 
                             chunksize=in_chunk_size,
                             resampling=resampling, 
                             dtype=in_dtype, 
                             fill_value=fill_value, 
                             rescale=in_rescale)

Converting data into dask array.
Converted data successfully.


In [6]:
in_cell_align = 'Top-left'

# prepare alignment type
cell_align = arc.prepare_cell_align_type(in_cell_align)

# generate coordinates and dimensions from metadata
coords, dims = cog.build_coords(feats=feats,
                                assets=assets, 
                                meta=meta,
                                pix_loc=cell_align)

Creating dataset coordinates and dimensions.
Created coordinates and dimensions successfully.


In [7]:
# build final xarray data array
ds_name = 'stac-' + dask.base.tokenize(darray)
ds = xr.DataArray(darray,
                  coords=coords,
                  dims=dims,
                  name=ds_name
                  )

In [8]:
# comvert to cleaner xarray dataset
ds = ds.to_dataset(dim='band')

# append attributes onto dataset
ds = cog.build_attributes(ds=ds,
                          meta=meta, 
                          fill_value=fill_value, 
                          collections=collections, 
                          slc_off=in_slc_off, 
                          bbox=bbox,
                          resampling=in_resampling)

Preparing and appending attributes to dataset.
Attributes appended to dataset successfully.


In [ ]:
# get list of dataset vars and iterate compute on each
for counter, data_var in enumerate(list(ds.data_vars), start=1):

    # start clock
    start = time.time()

    # update progress bar
    print('Downloading band: {}...'.format(data_var))

    # compute!
    ds[data_var] = ds[data_var].compute()

    # notify time 
    duration = round(time.time() - start, 2)
    print('Band: {} took: {}s to download.'.format(data_var, duration)) 

In [ ]:
# try doing all in one go
# start clock
start = time.time()

# update progress bar
print('Downloading dataset...')

# compute!
ds = ds.compute()

# notify time 
duration = round(time.time() - start, 2)
print('Dataset took: {}s to download.'.format(duration)) 

In [11]:
import concurrent

def compute_dataarray(var):
    return ds[var].compute()

In [12]:
# start clock
start = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
     result = executor.map(compute_dataarray, [var for var in ds.data_vars])
        
# show me time
duration = round(time.time() - start, 2)
print('Dataset took: {}s to download.'.format(duration)) 

Dataset took: 260.77s to download.


['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir_1', 'nbart_swir_2', 'nbart_swir_3', 'fmask']